In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import csv
from datetime import datetime, timedelta
# 1. 날짜 및 초기 설정
ds = "2022.11.07"  # 시작 날짜 (YYYY.MM.DD 형식)
duration = 7       # 기본 기간 (days)
min_duration = 0   # 최소 기간
# 오늘 날짜 계산
today = datetime.now().strftime("%Y.%m.%d")
# 날짜 형식 변환 함수
def add_days_to_date(date_str, days):
    date_obj = datetime.strptime(date_str, "%Y.%m.%d")
    new_date_obj = date_obj + timedelta(days=days)
    return new_date_obj.strftime("%Y.%m.%d")
while True:
    try:
        # 종료 조건: ds가 오늘 날짜를 초과하면 중단
        if datetime.strptime(ds, "%Y.%m.%d") > datetime.strptime(today, "%Y.%m.%d"):
            print("시작 날짜가 오늘 날짜를 초과했습니다. 크롤링을 종료합니다.")
            break
        # 2. 종료 날짜 계산
        de = add_days_to_date(ds, duration)
        # Selenium 설정
        options = webdriver.ChromeOptions()
        options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
        driver = webdriver.Chrome(options=options)
        # url = f"https://search.naver.com/search.naver?where=news&query=%EA%B8%88%EB%A6%AC&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds={ds}&de={de}&docid=&related=0&mynews=1&office_type=1&office_section_code=3&news_office_checked=1018&nso=so%3Ar%2Cp%3Afrom20081002to20081002&is_sug_officeid=0&office_category=0&service_area=0"
        url =f"https://search.naver.com/search.naver?where=news&query=%EA%B8%88%EB%A6%AC&sm=tab_opt&sort=2&photo=0&field=0&pd=3&ds={ds}&de={de}&docid=&related=0&mynews=1&office_type=1&office_section_code=3&news_office_checked=1014&nso=so%3Ar%2Cp%3Afrom20081002to20081002&is_sug_officeid=0&office_category=0&service_area=0"
        driver.get(url)
        # 무한 스크롤 처리
        SCROLL_PAUSE_TIME = 2  # 스크롤 간 대기 시간
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(SCROLL_PAUSE_TIME)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        # BeautifulSoup으로 HTML 파싱
        soup = BeautifulSoup(driver.page_source, "html.parser")
        articles = soup.select(".news_wrap")  # 기사 목록 선택자
        # 데이터 수집
        results = []
        for article in articles:
            try:
                url_tag = article.select_one("a.info[href^='https://n.news.naver.com']")
                if url_tag:
                    url = url_tag["href"]
                    date_tag = url_tag.find_previous_sibling("span", class_="info")
                    date = date_tag.text.strip() if date_tag else "날짜 없음"
                    results.append({"url": url, "date": date})
            except Exception as e:
                print(f"Error parsing article: {e}")
        # 결과 저장 (CSV 파일)
        output_filename = f"naver_news_{ds.replace('.', '')}_{de.replace('.', '')}_Financial_News.csv"
        with open(output_filename, "w", encoding="utf-8-sig", newline="") as f:
            writer = csv.DictWriter(f, fieldnames=["url", "date"])
            writer.writeheader()
            writer.writerows(results)
        print(f"크롤링 완료! 총 수집된 기사: {len(results)}")
        print(f"결과가 저장되었습니다: {output_filename}")
        driver.quit()
        # 다음 실행을 위한 ds 업데이트 (de + 1)
        ds = add_days_to_date(de, 1)
    except Exception as e:
        print(f"오류 발생: {e}")
        # duration을 줄여서 재시도 (최소값은 min_duration)
        duration -= 1
        if duration < min_duration:
            print("최소 기간에 도달하여 크롤링을 중단합니다.")
            break
    finally:
        try:
            driver.quit()
        except:
            pass

크롤링 완료! 총 수집된 기사: 231
결과가 저장되었습니다: naver_news_20221107_20221114_Financial_News.csv
크롤링 완료! 총 수집된 기사: 202
결과가 저장되었습니다: naver_news_20221115_20221122_Financial_News.csv
크롤링 완료! 총 수집된 기사: 182
결과가 저장되었습니다: naver_news_20221123_20221130_Financial_News.csv
크롤링 완료! 총 수집된 기사: 201
결과가 저장되었습니다: naver_news_20221201_20221208_Financial_News.csv
크롤링 완료! 총 수집된 기사: 177
결과가 저장되었습니다: naver_news_20221209_20221216_Financial_News.csv
크롤링 완료! 총 수집된 기사: 180
결과가 저장되었습니다: naver_news_20221217_20221224_Financial_News.csv
크롤링 완료! 총 수집된 기사: 168
결과가 저장되었습니다: naver_news_20221225_20230101_Financial_News.csv
크롤링 완료! 총 수집된 기사: 198
결과가 저장되었습니다: naver_news_20230102_20230109_Financial_News.csv
크롤링 완료! 총 수집된 기사: 244
결과가 저장되었습니다: naver_news_20230110_20230117_Financial_News.csv
크롤링 완료! 총 수집된 기사: 176
결과가 저장되었습니다: naver_news_20230118_20230125_Financial_News.csv
크롤링 완료! 총 수집된 기사: 259
결과가 저장되었습니다: naver_news_20230126_20230202_Financial_News.csv
크롤링 완료! 총 수집된 기사: 169
결과가 저장되었습니다: naver_news_20230203_20230210_Financial_News.csv
크롤링 

In [ ]:
import pandas as pd
import glob

# CSV 파일 목록 불러오기
file_list = glob.glob("C:/Users/wosle/OneDrive/Desktop/Bok_Projet_woslek/e_today_2011/*.csv")  # 예: "C:/data/*.csv"

# 모든 파일을 읽어서 하나로 합치기
df = pd.concat([pd.read_csv(file) for file in file_list], ignore_index=True)

# 결과 저장
df.to_csv("merged.csv", index=False)